<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Notebook Help

| What                  |Keyboard Shortcut |
|:-|:-|
| Run a cell            |Shift+Enter       |
| Restart python kernel |Esc+0+0           |
| Interrupt kernel      |Esc+i+i           |

In [4]:
import copy
import imp
import pandas as pd
import yaml

In [5]:
import araalictl
import api
import meta_policy_gcp
import meta_policy_metadata_svc
import meta_policy_araali
import meta_policy_k8s
import meta_policy_ubuntu
import meta_policy_docker
import meta_policy_aws
import meta_policy_lacework

imp.reload(araalictl)
imp.reload(api)
imp.reload(meta_policy_gcp)
imp.reload(meta_policy_metadata_svc)
imp.reload(meta_policy_araali)
imp.reload(meta_policy_k8s)
imp.reload(meta_policy_ubuntu)
imp.reload(meta_policy_docker)
imp.reload(meta_policy_aws)
imp.reload(meta_policy_lacework)

Adding meta-policies to runner
Adding meta-policies to runner
Adding meta-policies to runner
Adding meta-policies to runner
Adding meta-policies to runner
Adding meta-policies to runner
Adding meta-policies to runner
Adding meta-policies to runner
Adding meta-policies to runner
Adding meta-policies to runner
Adding meta-policies to runner
Adding meta-policies to runner
Adding meta-policies to runner
Adding meta-policies to runner
Adding meta-policies to runner
Adding meta-policies to runner


<module 'meta_policy_lacework' from '/Users/abhinavtata/opensrc/api/python/meta_policy_lacework.py'>

In [6]:
import meta_policy
imp.reload(meta_policy)

Adding meta-policies to runner
Adding meta-policies to runner


<module 'meta_policy' from '/Users/abhinavtata/opensrc/api/python/meta_policy.py'>

# Specific App

In [8]:
app = api.App("nightly", "bendvm")

In [ ]:
api.LinkTable(app.iterlinks())

In [ ]:
app.commit()

## Zoom into link

In [ ]:
%%time
flow_page = app.links[50].zoom()

In [ ]:
api.Table(flow_page.links)

In [ ]:
next_page = flow_page.next_page()

## Relocation

In [ ]:
app = api.App("nightly", "dmzvm")

In [ ]:
api.Table(app.link_stats(all=True, only_new=True))

In [ ]:
app2 = app.relocate("prod", "dmzvm")

In [ ]:
yaml.dump(app.to_data(), open("file1.yaml", "w"))
yaml.dump(app2.to_data(), open("file2.yaml", "w"))
print("Run in terminal:\n\t tkdiff file1.yaml file2.yaml")

In [ ]:
api.LinkTable(
    api.MetaPolicyRunner(
        meta_policy.NightlyToProd, meta_policy.AcceptAllDefined)
    .run(app2.iterlinks()).review(todo=False)
)

In [ ]:
app2.commit()

# Runtime Review

## Specific zones and apps only

In [ ]:
api.Runtime.zone2apps = {                                                               
    "dev": ["cassandra"],                                                   
    "ops": ["cassandra"],                                                   
    "prod": ["dmzvm", "bendvm"],                                            
    "nightly": ["dmzvm", "bendvm"],                                         
    "prod-k8s": ["k8s", "kube-system", "monitoring", "prod-araali-operator", "prod-bend"],
    "nightly-k8s": ["k8s", "kube-system", "monitoring", "prod-araali-operator", "prod-bend"],
}
run = api.Runtime()

In [ ]:
%%time
run = run.refresh()

## All Zones and Apps

In [ ]:
api.Runtime.zone2apps = None
run = api.Runtime()

In [ ]:
%%time
run = run.refresh()

In [ ]:
api.Table(api.Runtime.get_zone_apps(hard=False))

## Summary Stats

In [ ]:
api.Table(run.stats(all=True))

## All the links

In [ ]:
api.LinkTable(run.iterlinks())

In [ ]:
for zone in run.iterzones():
    for app in zone.iterapps():
        for link in app.iterlinks():
            print(link)

## Link Science

### Changes ready to commit

In [ ]:
runlinks = list(run.iterlinks())

### Simulate changes iteratively

In [ ]:
runlinks = copy.deepcopy(list(run.iterlinks()))

### Alert Count

In [ ]:
api.Table(run.link_stats(all=False, only_new=True, runlink=runlinks))

In [ ]:
api.Table(run.dns_stats(only_new=True, runlink=runlinks))

In [ ]:
api.Table(run.server_stats(only_new=True, runlink=runlinks))

# Templates

## Convert a link to template

In [ ]:
app = api.App("app-nightly", "app-nightly-bend")

In [ ]:
template = app.links[0].template(show=True)

### Modify Template

In [ ]:
template.modify(changes=[api.TemplateChanges("client", "netmask", 0), 
                         api.TemplateChanges("server", "zone", "", delete=True),
                         api.TemplateChanges("server", "app", "", delete=True),
                         api.TemplateChanges("server", "binary_name", "/usr/bin/python.*", replace=True)
                        ])

In [ ]:
template.show()

### Accept Template

In [ ]:
# NOTE: set use=True to start using the templates and idxs to choose which template.
template.accept(use=False, idxs=[0])

### List Templates

In [ ]:
my_templates = api.Templates()

In [ ]:
my_templates.show()
# NOTE: templates change be modified/accepted as well.

### Delete Template

In [ ]:
my_templates.delete(idxs=[8])

# Meta Policies and Suggestions

In [ ]:
api.LinkTable(
    api.mpr.run(
        api.LinkTable(runlinks,
                      api.f.neg(api.f.state("DEFINED_POLICY")),
                     ).links, show_all=False,
    ).review(todo=False)
)

## Pending Ones

In [ ]:
api.LinkTable(runlinks,
              api.f.new_state(None),
              #lambda x: x.get("meta_policy", None) is not None,
              #api.f.state("BASELINE_ALERT"),
              #api.f.endpoint("subnet", ["169.254.169.254", "127.0.0.1"], who="server"),
              #api.f.endpoint("process", "dockerd", who="client"),
              #api.f.endpoint("dns_pattern", "googleapis.com"),
              #api.f.endpoint("zone", "nightly-k8s"), api.f.endpoint("app", "^monitoring\."),
              #api.f.any(api.f.same_zone, api.f.type(["NAI", "NAE"])),
              #api.f.neg(api.f.endpoint("process", "grafana-server", who="client")),
              #api.f.neg(api.f.same_zone), api.f.type(["AEG", "AIN"]),
              #api.f.same_zone, api.f.type(["AEG", "AIN"]),
              #api.f.neg(api.f.type(["AEG", "AIN"]), api.f.same_zone),
              api.f.type("NAE"), #api.f.same_zone,
              #api.f.perimeter, #api.f.endpoint("process", ["sshd"])
              #api.f.neg(api.f.perimeter),
              #api.f.neg(api.f.server_non_ip),
              #api.f.server_non_ip,
              #api.f.inactive_open_ports
             )

In [ ]:
_.dump(0)

In [ ]:
_.meta_policy()

In [ ]:
_.snooze()

In [ ]:
_.accept()

In [ ]:
_.deny()

In [ ]:
api.Table(api.za_stats(_.links, only_new=False))

In [ ]:
api.Table(_.dns_stats(only_new=True))

In [ ]:
api.LinkTable(run.review())

In [ ]:
%%time
api.Table(run.commit())

## Trying a new Meta Policy

In [ ]:
class MpTest:
    policies = [
        api.AcceptLink(filters=[                                                
                api.f.type("NAE"),
                api.f.endpoint("process", "dockerd", who="client"),             
                api.f.endpoint("dns_pattern", [":.*.docker.io:"], who="server"),
            ], changes=[                                                        
                ("server", "dns_pattern", ":.*\.docker\.io:"),                  
            ]),
    ]

In [ ]:
api.LinkTable(api.MetaPolicyRunner(MpTest).run(runlinks).review(MpTest))

# Documentation

## Meta-Policy HowTo

Meta policy are policies that are in turn used to accept policies (with or without changes and transformations). Araali policies are least permissive and precise, but they can be auto-accepted using patterns encoded into these meta policies. While accepting the policy, the endpoint parameters can be changed/rewritten (for client or server params in the policy).

Using the info displayed for a link in Alert state, create a meta policy using the following guidelines depending on the link type:
 * NAE
   * First find out if the process is already in some other meta-policy block you may have defined
     * It might be simple enough to add another dns into an existing meta policy (first example).
     * If there is change to the default suggested policy, you should not use a list for dns_pattern, and instead have one block per dns_pattern that is rewritten
       * In the first example below, if you put \*.pythonhosted.\*, it will end up using that even for links that match pypi.org
       * Instead use the separate block as shown in the second example
   * Else define a new block.
     * Typically you would want to have client app, process, and dns_pattern (to keep things zone independent) in the link filters.
<pre>        
    AcceptLink(filters=[                                                    
            f.type("NAE"),
            f.endpoint("app", "cassandra"),                                 
            f.endpoint("process", "/usr/bin/pip3", who="client"),           
            f.endpoint("dns_pattern", [":pypi.python.org:",                 
                                       ":pypi.org:",                        
                                       ":files.pythonhosted.org:"], who="server"),
        ], changes=[                                                        
        ]),
    AcceptLink(filters=[                                                    
            f.endpoint("process", ["/usr/bin/yum", "amazon_linux_extras"], who="client"),
            f.endpoint("dns_pattern", ":amazonlinux.us-west-2.amazonaws.com:", who="server"),
        ], changes=[                                                        
            ("server", "dns_pattern", ":amazonlinux\..*\.amazonaws\.com:"), 
        ]),
</pre>            
  * NAI
    * see if the service really needs public internet exposure. These are easy to spot with the pre-defined perimeter filter.
    * Define a block with f.perimeter filter and the process and app that needs perimeter exposure
    * Araali by default exposes these to the world, but you can restrict/change it to a subnet instead.
<pre>        
    AcceptLink(filters=[                                                    
            f.perimeter,                                                    
            f.endpoint("zone", ["prod", "nightly", "dev", "ops", "nightly-k8s"], who="server"),
            f.endpoint("app", ["dmzvm", "cassandra", "k8s"], who="server", flags=re.IGNORECASE), 
            f.endpoint("process", ["sshd", "haproxy"], who="server", flags=re.IGNORECASE),
        ], changes=[
            ("client", "network", "192.168.0.0"),
            ("client", "mask", 16),
        ]),
</pre>            
            
  * INT
    * There is no need to specify zone, because internal implies it by default
<pre>
    AcceptLink(filters=[                                                    
        f.type("INT"),
        f.endpoint("app", "monitoring.grafana.grafana"),          
        f.endpoint("process", "grafana-server", who="client"),          
        f.endpoint("process", "prometheus", who="server"),              
    ], changes=[                                                        
    ]),
</pre>            
  * AIN
    * These can be ommitted
      * Covering AEG automatically covers this category.
  * AEG
    * If both client and server should be in the same zone, use a same_zone filter, else both client and server zones can be explicitly specified (cross zone case). The example below uses same_zone.
<pre>
    AcceptLink(filters=[                                                    
            f.same_zone,                                                    
            f.type("AEG"),
            f.endpoint("app", "dmzvm", who="client"),                       
            f.endpoint("process", "/var/lib/haproxy/healthcheck.py", who="client"),
            f.endpoint("app", "bendvm.bend.backend", who="server"),         
            f.endpoint("process", "prometheus", who="server"),              
        ], changes=[                                                        
        ]),
</pre>            

# Installing Python Packages into Notebook

In [ ]:
# This is needed only one time, to make sure all dependent packages are installed
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install pyyaml
!{sys.executable} -m pip install oyaml
!{sys.executable} -m pip install pandas